In [1]:
##complex data와 똑같이 40ms로 parsing했음 데이터 4배라고 생각하고 뻥튀기시켜봄 정확도는 10ms와 비슷하게 나옴

import csv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import sklearn
from sklearn import svm
from datetime import datetime
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score 
import glob


#filename queue
filename_queue=glob.glob('./rawdata/data[1-5].csv')
#Create return path
#return_path='./parsed/data.csv'
#create_file=open(return_path,'w')

In [2]:
#Parsing data

training_set=[]
test_set=[]
training_label=[]
test_label=[]
index=[] 
label_count=np.zeros(16)
traininglabel_count=np.zeros(16)
testlabel_count=np.zeros(16)

#return_file=open(return_path,'a')
#writer=csv.writer(return_file)
#writer.writerow()
n_data=[]

for filename in filename_queue:
    file=open(filename, newline='')
    
    reader=csv.reader(file)
    header=next(reader)
    
    for row in reader:
        row[4:]=[float(i) for i in row[4:]]
        date=datetime.strptime(row[0],'%Y/%m/%d')
        msec=row[1:3]
        labl=int(row[3])
        if labl not in index:
            label_count[labl]+=1
            for x in range(4):
                X_acc=list(row[4*i+4+x] for i in range(150))
                Y_acc=list(row[4*i+604+x] for i in range(150))
                Z_acc=list(row[4*i+1204+x] for i in range(150))
                X_gyro=list(row[4*i+1804+x] for i in range(150))
                Y_gyro=list(row[4*i+2404+x] for i in range(150))
                Z_gyro=list(row[4*i+3004+x] for i in range(150))

                window=np.array([[X_acc[0:25],X_acc[25:50],X_acc[50:75],X_acc[75:100],X_acc[100:125],X_acc[125:150]],
                    [Y_acc[0:25],Y_acc[25:50],Y_acc[50:75],Y_acc[75:100],Y_acc[100:125],Y_acc[125:150]],
                    [Z_acc[0:25],Z_acc[25:50],Z_acc[50:75],Z_acc[75:100],Z_acc[100:125],Z_acc[125:150]],
                    [X_gyro[0:25],X_gyro[25:50],X_gyro[50:75],X_gyro[75:100],X_gyro[100:125],X_gyro[125:150]],
                    [Y_gyro[0:25],Y_gyro[25:50],Y_gyro[50:75],Y_gyro[75:100],Y_gyro[100:125],Y_gyro[125:150]],
                    [Z_gyro[0:25],Z_gyro[25:50],Z_gyro[50:75],Z_gyro[75:100],Z_gyro[100:125],Z_gyro[125:150]]])

                #5 features  what else?2
                window_mean=window.mean(axis=-1)
                window_stddev=window.std(axis=-1)
                window_median=np.median(window,axis=-1)
                window_percent25=np.percentile(window,25,axis=-1)
                window_percent75=np.percentile(window,75,axis=-1)

                window_feature=np.array([[window_mean],[window_stddev],[window_median],[window_percent25],[window_percent75]])
                window_feature=window_feature.reshape(180)


                #for cross validation
                training_set.append(window_feature)
                training_label.append(labl)


In [3]:
print(np.shape(training_set))

(1400, 180)


In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

X=training_set
y=training_label

clf=RandomForestClassifier()
param_grid={
 'n_estimators':[100,120,180,200]
   # 'max_features':['auto','sqrt','log2'],
    #'max_depth':[10,12,None]
    #'criterion':['gini','entropy']
}

grid=GridSearchCV(estimator=clf,param_grid=param_grid,cv=10,scoring='accuracy')

In [11]:
grid.fit(X,y)

/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 4 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_start=False),
             ii

In [12]:
grid.cv_results_

{'mean_fit_time': array([0.64107876, 0.81865051, 1.2547591 , 1.34238887]),
 'std_fit_time': array([0.02503468, 0.05360534, 0.06013051, 0.07511516]),
 'mean_score_time': array([0.00781858, 0.01016967, 0.0140393 , 0.01517713]),
 'std_score_time': array([0.0016428 , 0.00279391, 0.00233618, 0.00282394]),
 'param_n_estimators': masked_array(data=[100, 120, 180, 200],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'n_estimators': 100},
  {'n_estimators': 120},
  {'n_estimators': 180},
  {'n_estimators': 200}],
 'split0_test_score': array([0.81512605, 0.78151261, 0.79831933, 0.79831933]),
 'split1_test_score': array([0.93277311, 0.94957983, 0.95798319, 0.94957983]),
 'split2_test_score': array([0.96581197, 0.94017094, 0.94871795, 0.93162393]),
 'split3_test_score': array([0.88888889, 0.86324786, 0.87179487, 0.85470085]),
 'split4_test_score': array([0.92982456, 0.92982456, 0.92982456, 0.92982456]),
 'split5_test_score': array(

In [13]:
grid.best_params_

{'n_estimators': 180}

In [14]:
RF=RandomForestClassifier(120).fit(X,y)

In [15]:
clf1=RandomForestClassifier(180)
scores=cross_val_score(clf1,X,y,cv=10,scoring='accuracy')
print(scores.mean())

/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 4 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


0.8824534880800341


In [16]:
from joblib import dump, load
import pickle

filename='./rf.sav'
pickle.dump(RF,open(filename,'wb'))
dump(RF,'rf.joblib')

['rf.joblib']

In [17]:
#SVM
#prepare the data
#prepare the model
clf = sklearn.svm.SVC(gamma='scale',tol=0.1)
scores=cross_val_score(clf,X,y,cv=10,scoring='accuracy')
print(scores.mean())


/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 4 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


0.7856916637489739


In [18]:
param_grid={
 #'kernel':['linear','poly','rbf','sigmoid'], rbf to be best acc
    'C':[0.001,0.01,0.1,1,10],
    'gamma' : [0.001, 0.01, 0.1, 1]

}

grid=GridSearchCV(estimator=clf,param_grid=param_grid,cv=10,scoring='accuracy')

In [19]:
grid.fit(X,y)

/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 4 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.1, verbose=False),
             iid='warn', n_jobs=None,
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10],
                         'gamma': [0.001, 0.01, 0.1, 1]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=0)

In [20]:
grid.best_params_

{'C': 10, 'gamma': 0.001}

In [21]:
grid.cv_results_

{'mean_fit_time': array([0.38621025, 0.36506433, 0.34144514, 0.35732546, 0.37990196,
        0.36316302, 0.36580777, 0.36327689, 0.39837914, 0.3736824 ,
        0.35980639, 0.35884345, 0.51047113, 0.56690602, 0.53920162,
        0.5451309 , 0.5013417 , 0.55420961, 0.5409827 , 0.54127841]),
 'std_fit_time': array([0.01850753, 0.01975851, 0.01719987, 0.0170773 , 0.02152598,
        0.02216406, 0.02223093, 0.02049109, 0.01233073, 0.0216461 ,
        0.01474867, 0.01875789, 0.02915742, 0.03807799, 0.03477942,
        0.03204563, 0.01583691, 0.01870678, 0.01695084, 0.01540926]),
 'mean_score_time': array([0.03097653, 0.0315886 , 0.02756238, 0.03082104, 0.0322603 ,
        0.03083332, 0.03110309, 0.03060074, 0.03469377, 0.03216541,
        0.02947938, 0.03038328, 0.03019207, 0.03058653, 0.03166723,
        0.03029504, 0.02821579, 0.03033664, 0.0310971 , 0.02822978]),
 'std_score_time': array([0.00415337, 0.00298154, 0.00083856, 0.00414126, 0.00387971,
        0.0028207 , 0.00463454, 0.003944

In [22]:
clf1=sklearn.svm.SVC(gamma='scale',tol=0.1)
scores=cross_val_score(clf1,X,y,cv=10,scoring='accuracy')
print(scores.mean())

/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 4 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


0.7856916637489739


In [23]:
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
bagging = BaggingClassifier(KNeighborsClassifier(),max_samples=0.5, max_features=0.5)

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier

ETC=ExtraTreesClassifier(300)
DTC=DecisionTreeClassifier()








In [24]:
param_grid={
 'n_estimators':[100,200,300],
   # 'max_features':['auto','sqrt','log2'],
    'max_depth':[10,14,None]
    #'criterion':['gini','entropy']
}

grid=GridSearchCV(estimator=ETC,param_grid=param_grid,cv=10,scoring='accuracy')

In [25]:
grid.fit(X,y)


/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 4 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=ExtraTreesClassifier(bootstrap=False, class_weight=None,
                                            criterion='gini', max_depth=None,
                                            max_features='auto',
                                            max_leaf_nodes=None,
                                            min_impurity_decrease=0.0,
                                            min_impurity_split=None,
                                            min_samples_leaf=1,
                                            min_samples_split=2,
                                            min_weight_fraction_leaf=0.0,
                                            n_estimators=300, n_jobs=None,
                                            oob_score=False, random_state=None,
                                            verbose=0, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'max_depth': [10, 14, No

In [26]:
grid.cv_results_

{'mean_fit_time': array([0.14025004, 0.3196059 , 0.4143013 , 0.18607178, 0.31325231,
        0.48845336, 0.19496381, 0.30947039, 0.47086363]),
 'std_fit_time': array([0.00618668, 0.04483624, 0.01412918, 0.05484848, 0.01945254,
        0.04089591, 0.0421529 , 0.00647546, 0.03316024]),
 'mean_score_time': array([0.0065475 , 0.01415203, 0.01916993, 0.00861475, 0.01364317,
        0.0231262 , 0.00988085, 0.01345694, 0.02001011]),
 'std_score_time': array([0.00015982, 0.00279083, 0.00171597, 0.00315099, 0.00064988,
        0.00553538, 0.00463208, 0.00025834, 0.00188347]),
 'param_max_depth': masked_array(data=[10, 10, 10, 14, 14, 14, None, None, None],
              mask=[False, False, False, False, False, False, False, False,
                    False],
        fill_value='?',
             dtype=object),
 'param_n_estimators': masked_array(data=[100, 200, 300, 100, 200, 300, 100, 200, 300],
              mask=[False, False, False, False, False, False, False, False,
                    Fals

In [27]:
test_pred_proba2=ETC.predict_proba(test_set)
array=np.array(test_pred_proba2)
newpred3=np.zeros(len(test_pred4))
for x in range(len(test_pred_proba2)):
    if(np.max(test_pred_proba2[x])>0.2):
        newpred3[x]=test_pred4[x]
    else:
        newpred3[x]=-1
        i=0
        #print(np.shape(np.argmax(test_pred_proba[x])))
print(np.shape(newpred3))
print(newpred3)
print(accuracy_score(newpred3,test_label))

NotFittedError: This ExtraTreesClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.

In [77]:
from joblib import dump, load
import pickle

filename='./rf.sav'
pickle.dump(RF,open(filename,'wb'))
dump(RF,'rf.joblib')

['rf.joblib']